In [1]:
! pip install langchain


     -------------------------------------- 789.1/789.1 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 92.7/92.7 kB 5.2 MB/s eta 0:00:00
     ---------------------------------------- 90.0/90.0 kB 1.7 MB/s eta 0:00:00
  Using cached tenacity-8.2.2-py3-none-any.whl (24 kB)
     ---------------------------------------- 49.1/49.1 kB 1.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.0.1
    Uninstalling tenacity-8.0.1:
      Successfully uninstalled tenacity-8.0.1
  Attempting uninstall: numexpr
    Found existing installation: numexpr 2.8.3
    Uninstalling numexpr-2.8.3:
      Successfully uninstalled numexpr-2.8.3


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\SinjiniGhosh\\anaconda3\\Lib\\site-packages\\~umexpr\\interpreter.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
! pip install transformers

In [3]:

!pip install -q langchain==0.0.150 pypdf pandas matplotlib tiktoken textract transformers openai faiss-cpu

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.2.2 requires pyqt5<5.13, which is not installed.
spyder 5.2.2 requires pyqtwebengine<5.13, which is not installed.
prophet 1.0.1 requires cmdstanpy==0.9.68, which is not installed.
flair 0.12.1 requires huggingface-hub==0.10.0, but you have huggingface-hub 0.13.3 which is incompatible.
conda-repo-cli 1.0.20 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.20 requires nbformat==5.4.0, but you have nbformat 5.5.0 which is incompatible.
arviz 0.11.2 requires typing-extensions<4,>=3.7.4.3, but you have typing-extensions 4.3.0 which is incompatible.
anaconda-client 1.11.0 requires six>=1.15.0, but you have six 1.12.0 which is incompatible.


In [4]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from transformers import GPT2TokenizerFast
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

In [ ]:
import openai
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

In [10]:

# Split by pages 
loader = PyPDFLoader("BTS_chapter2_document.pdf")
pages = loader.load_and_split()
print(pages[0])


chunks = pages



page_content='Subject: Proof Album by BTS.  \nProof is the first anthology album released by South Korean group BTS, on June 10, 2022, through Big Hit \nMusic. The 3 -disc project comprises several of the band\'s singles through the years; a selection of \ndiscography "favorites " chosen by the band members; and various demos and previously unreleased \ntracks. It also includes five new songs: the album\'s lead single "Yet to Come (The Most Beautiful \nMoment)", "Run BTS", "For Youth", "Quotation Mark", and " 애매한 \xa0사이 "\xa0(Young Love). The album sold \nover 2 million copies worldwide on its opening day and topped the charts in 18 territories, including \nAustralia, Germany, Japan, South Korea, the United Kingdom, and the United States. It has been certified \ngold in France, Ne w Zealand, and Poland; double platinum in Japan; and 3× Million in South Korea. Per a \nsubsequent notice posted on Weverse and shared via Twitter, the three -disc album comprises tracks \nspanning the past n

In [8]:
# Advanced method - Split by chunk

# Step 1: Convert PDF to text
import textract
doc = textract.process("BTS_chapter2_document.pdf")

# Step 2: Save to .txt and reopen (helps prevent issues)
with open('BTS_chapter2_document.txt', 'w') as f:
    f.write(doc.decode('utf-8'))

with open('BTS_chapter2_document.txt', 'r') as f:
    text = f.read()

# Step 3: Create function to count tokens
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

def count_tokens(text: str) -> int:
    return len(tokenizer.encode(text))

# Step 4: Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 512,
    chunk_overlap  = 24,
    length_function = count_tokens,
)

chunks = text_splitter.create_documents([text])

ShellError: The command `pdftotext BTS_chapter2_document.pdf -` failed with exit code 127
------------- stdout -------------
------------- stderr -------------


In [11]:
# Get embedding model
embeddings = OpenAIEmbeddings()

# Create vector database
db = FAISS.from_documents(chunks, embeddings)

In [12]:
# Check similarity search is working
query = "Tell me about RM's album Indigo."
docs = db.similarity_search(query)
docs[0]

Document(page_content='Subject: Indigo Album by RM of BTS.  \n Indigo  is the debut  studio album  by South Korean rapper  RM, released on December 2, 2022, \nthrough  Big Hit Music. The album marks the rapper\'s first full -length body of work since  Mono  (2018) \nand serves as a documentation or archive of his late  twenties. Comprising 10 tracks, it includes \nappearances by  Erykah B adu,  Anderson .Paak,  Tablo  of Epik High,  Kim Sa -wol, Paul \nBlanco,  Mahalia,  Colde, Youjeen of  Cherry Filter, and  Park Ji -yoon. The ninth track, "Wild Flower", a \ncollaboration with Youjeen, was released alongside the album as its lead single, together with a n \naccompanying music video. The album peaked at number two in South Korea; number three in \nLithuania, Portugal, and the United States; and number four in Japan. It was certified double platinum by \nthe Korea Music Content Association and has sold over 700,0 00 copies domestically. It is the highest -\ncharting album by a Korean so

In [13]:
# Create QA chain to integrate similarity search with user queries (answer query from knowledge base)

chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff")

query = "Who is Agust D?"
docs = db.similarity_search(query)

chain.run(input_documents=docs, question=query)

' Agust D is the stage name of South Korean rapper Suga of the boy band BTS.'

## Chatbot.

In [14]:
from IPython.display import display
import ipywidgets as widgets

# Create conversation chain that uses our vectordb as retriver, this also allows for chat history management
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0.1), db.as_retriever())

In [20]:
chat_history = []

def on_submit(_):
    query = input_box.value
    input_box.value = ""
    
    if query.lower() == 'exit':
        print("Thank you for using the chatbot!")
        return
    
    result = qa({"question": query, "chat_history": chat_history})
    chat_history.append((query, result['answer']))
    
    display(widgets.HTML(f'<b>User:</b> {query}'))
    display(widgets.HTML(f'<b><font color="blue">Chatbot:</font></b> {result["answer"]}'))

print("Ask the chatbot any question about BTS music albums released in 2022 and 2023. Type 'exit' to stop.")

input_box = widgets.Text(placeholder='Please enter your question:')
input_box.on_submit(on_submit)

display(input_box)

Ask the chatbot any question about BTS music albums released in 2022 and 2023. Type 'exit' to stop.


Text(value='', placeholder='Please enter your question:')

HTML(value='<b>User:</b> What important message is Agust D trying to convey with his album D-Day?')

HTML(value='<b><font color="blue">Chatbot:</font></b>  Agust D is trying to convey the idea of pushing forward…

HTML(value='<b>User:</b> How many songs does the album have?')

...

HTML(value='<b>User:</b> What is the name of the second song?')

HTML(value='<b><font color="blue">Chatbot:</font></b>  The second song on Agust D\'s album D-Day is "Haegeum".…

HTML(value='<b>User:</b> What is that song about?')

HTML(value='<b><font color="blue">Chatbot:</font></b>  "Haegeum" is a heavy hip hop song whose title is a word…

Thank you for using the chatbot!
